# Visualización de datos

##Limpieza de datos

En esta parte del trabajo analizaremos la base de datos con la que trabajaremos a lo largo del proyecto. Nuestro objetivo es comprender las características principales de los datos, como el número total de pacientes, la detección de valores atípicos (outliers), la distribución de pacientes según su localización y los meses en los que se registra una mayor afluencia a urgencias. Además, realizaremos ajustes en las columnas y añadiremos nuevas variables para refinar la base de datos con la que trabajaremos a lo largo del proyecto.

Este análisis preliminar nos permitirá obtener una visión clara y detallada de los datos, lo que será fundamental para extraer conclusiones significativas. A partir de este conocimiento, podremos avanzar hacia la etapa de modelado y realizar predicciones mediante el uso de algoritmos avanzados, como XGBoost.


In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from meteostat import Stations, Hourly
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import calendar

: 

####**1. Lectura del archivo.**

Como primer paso, cargamos el archivo .csv, mostramos los primeros datos y analizamos sus principales características. Las columnas con las que contamos son las siguientes:

* Fecha de atención: Indica el día, mes y año en que el paciente acudió a urgencias.
* Día de la semana: Nombre del día de la semana correspondiente a la fecha de atención.
* Nivel de triaje: Representa el nivel de urgencia de atención del paciente.
* Zona Básica de Salud: Zona de referencia médica asignada al paciente.
* Ámbito de procedencia: Clasificación del ámbito del paciente, ya sea urbano o rural.
* Hospital: Nombre del hospital donde fue atendido el paciente.
* Área: Área en la que se encuentra el hospital.
* Provincia: Provincia en la que se encuentra el hospital.
* Edad: Edad del paciente en años.
* Sexo: Sexo del paciente.

In [ ]:
df = pd.read_csv("urgencias-hospitalarias-atendidas.csv", sep=";")
df.head()

Lo primero que haremos será convertir las columnas Nivel de triaje y Edad a valores numéricos, asegurando que cualquier dato no válido se transforme en NaN, mientras que Zona Básica de Salud, Hospital, Área y Provincia serán convertidas en variables categóricas. Además, Sexo y Ámbito de procedencia se normalizarán a valores binarios y se transformarán en enteros, asignando NaN a los datos que no cumplan con los valores esperados.

In [ ]:
df[["Nivel de triaje", "Edad"]] = df[["Nivel de triaje", "Edad"]].apply(
    pd.to_numeric, errors="coerce"
)

columnas_categoricas = ["Zona Básica de Salud", "Hospital", "Área", "Provincia"]
df[columnas_categoricas] = df[columnas_categoricas].astype("string")

df["Sexo"] = df["Sexo"].map({"Hombre": 1, "Mujer": 0}).astype("Int64")
df["Ámbito de procedencia"] = (
    df["Ámbito de procedencia"].map({"Urbano": 1, "Rural": 0}).astype("Int64")
)

df.rename(columns={"Sexo": "Hombre", "Ámbito de procedencia": "Urbano"}, inplace=True)

print("Datos limpios y convertidos correctamente:")
print(df.dtypes)

####**2.Completar base de datos**



A continuación, añadiremos nuevas columnas que pueden ser de utilidad para nuestro análisis, como la indicación de días festivos, la temperatura y si ha llovido o no, entre otros factores relevantes. Antes de ello, comenzaremos creando una columna que contenga la fecha completa junto con el día, la hora, el año y el mes, lo que nos permitirá disponer de una estructura temporal más detallada para futuros estudios.

In [ ]:
print("Valores NaN en 'Fecha de atención':", df["Fecha de atención"].isna().sum())
print("Valores NaN en 'Hora':", df["Hora"].isna().sum())

print("Formatos únicos en 'Fecha de atención':")
print(df["Fecha de atención"].dropna().astype(str).str[:10].value_counts().head(10))

print("Valores problemáticos en 'Hora':")
print(
    df[~df["Hora"].astype(str).str.match(r"^\d{2}:\d{2}$", na=False)]["Hora"].unique()
)

In [ ]:
df["Fecha de atención"] = pd.to_datetime(df["Fecha de atención"], errors="coerce")

# Corregir "Hora" eliminando valores incorrectos
df["Hora"] = df["Hora"].astype(str)
df.loc[df["Hora"] == ":", "Hora"] = np.nan  # Reemplazar ":" por NaN
df.loc[~df["Hora"].str.match(r"^\d{2}:\d{2}$", na=False), "Hora"] = (
    np.nan
)  # Eliminar formatos inválidos
df["Hora"].fillna("00:00", inplace=True)  # Asigna "00:00" a los valores NaN restantes

# Asegurar que "Hora" tenga solo HH:MM (sin segundos)
df["Hora"] = pd.to_datetime(df["Hora"], format="%H:%M", errors="coerce").dt.strftime(
    "%H:%M"
)

df["Fecha completa"] = pd.to_datetime(
    df["Fecha de atención"].dt.strftime("%Y-%m-%d") + " " + df["Hora"],
    format="%Y-%m-%d %H:%M",
    errors="coerce",
)

print(
    f" Fechas inválidas detectadas después de la corrección: {df['Fecha completa'].isna().sum()}"
)
print(df[["Fecha de atención", "Día de la semana", "Hora", "Fecha completa"]].head())


Una vez creada la columna que contiene la fecha completa, transformamos el tiempo en una función sinusoidal para representar su circularidad temporal. Esto es especialmente útil en análisis de series temporales y modelos predictivos, ya que evita que el modelo interprete las variables temporales como lineales.  

Al aplicar funciones seno (`sin`) y coseno (`cos`), conseguimos que valores como la hora del día, el mes o el día del año conserven su periodicidad. De este modo, el modelo entiende que, por ejemplo, la hora 23 y la hora 0 están cerca, o que enero y diciembre son meses consecutivos en el ciclo anual. Esto mejora la precisión de los modelos y facilita la detección de patrones temporales en los datos.

In [ ]:
df["Hora del día"] = df["Fecha completa"].dt.hour  # 0-23
df["Día del año"] = df["Fecha completa"].dt.dayofyear  # 1-365
df["Mes del año"] = df["Fecha completa"].dt.month  # 1-12

# Transformación sinusoidal
df["Hora_sin"] = np.sin(2 * np.pi * df["Hora del día"] / 24)
df["Hora_cos"] = np.cos(2 * np.pi * df["Hora del día"] / 24)

df["Día_sin"] = np.sin(2 * np.pi * df["Día del año"] / 365)
df["Día_cos"] = np.cos(2 * np.pi * df["Día del año"] / 365)

df["Mes_sin"] = np.sin(2 * np.pi * df["Mes del año"] / 12)
df["Mes_cos"] = np.cos(2 * np.pi * df["Mes del año"] / 12)

Ahora identificaremos los días festivos en la comunidad de Castilla y León, añadiendo una columna binaria que tomará el valor 1 cuando la fecha corresponda a un festivo/domingo y 0 en caso contrario. Además, incorporaremos una nueva columna para indicar la estación del año en la que se encuentra cada fecha, lo que nos permitirá analizar posibles variaciones en los datos según la temporada.

In [ ]:
# Lista de festivos en Castilla y León
festivos = [
    "2021-01-01",
    "2021-01-06",
    "2021-04-01",
    "2021-04-02",
    "2021-04-23",
    "2021-05-01",
    "2021-08-16",
    "2021-10-12",
    "2021-11-01",
    "2021-12-06",
    "2021-12-08",
    "2021-12-25",
    "2022-01-01",
    "2022-01-06",
    "2022-04-14",
    "2022-04-15",
    "2022-04-23",
    "2022-05-02",
    "2022-08-15",
    "2022-10-12",
    "2022-11-01",
    "2022-12-06",
    "2022-12-08",
    "2022-12-26",
    "2023-01-02",
    "2023-01-06",
    "2023-04-06",
    "2023-04-07",
    "2023-05-01",
    "2023-07-25",
    "2023-08-15",
    "2023-10-12",
    "2023-11-01",
    "2023-12-06",
    "2023-12-08",
    "2023-12-25",
]
festivos = pd.to_datetime(festivos)

# Crear la columna 'Festivo' considerando también los domingos
df["Festivo"] = df["Fecha completa"].apply(
    lambda x: 1 if (x in festivos or x.weekday() == 6) else 0
)

df.head()


In [ ]:
def obtener_estacion(fecha):
    if fecha is pd.NaT:
        return None  # Si la fecha es NaN, devolver None

    mes = fecha.month
    dia = fecha.day

    if (mes == 12 and dia >= 21) or (mes in [1, 2]) or (mes == 3 and dia < 20):
        return "Invierno"
    elif (mes == 3 and dia >= 20) or (mes in [4, 5]) or (mes == 6 and dia < 21):
        return "Primavera"
    elif (mes == 6 and dia >= 21) or (mes in [7, 8]) or (mes == 9 and dia < 23):
        return "Verano"
    else:
        return "Otoño"


df["Estación del Año"] = df["Fecha completa"].apply(obtener_estacion)

A continuación, inicializaremos el geocodificador para obtener la latitud y longitud de cada hospital. Para optimizar este proceso, en lugar de extraer las coordenadas fila por fila, identificaremos los hospitales únicos presentes en el DataFrame y obtendremos sus coordenadas a partir de esa lista.

In [ ]:
geolocator = Nominatim(user_agent="hospital_locator")

hospitales_unicos = df["Hospital"].dropna().unique()


def obtener_coordenadas(hospital):
    try:
        location = geolocator.geocode(hospital + ", España", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except GeocoderTimedOut:
        return (None, None)


coordenadas_hospitales = {
    hospital: obtener_coordenadas(hospital) for hospital in hospitales_unicos
}

df_coordenadas = pd.DataFrame.from_dict(
    coordenadas_hospitales, orient="index", columns=["Latitud", "Longitud"]
).reset_index()
df_coordenadas.rename(columns={"index": "Hospital"}, inplace=True)

df = df.merge(df_coordenadas, on="Hospital", how="left")

df.head()

Comprobaremos si existe algún hospital en el que no hayamos obtenido las coordenadas, identificando aquellos registros que aún presentan valores nulos en las columnas de latitud y longitud.

In [ ]:
df["Latitud"].unique()

In [ ]:
df["Longitud"].unique()

In [ ]:
hospitales_sin_coordenadas = df[df[["Latitud", "Longitud"]].isna().any(axis=1)][
    ["Hospital", "Latitud", "Longitud"]
]

hospitales_sin_coordenadas["Hospital"].unique()

Hemos localizado las coordenadas faltantes a través de fuentes en internet y procederemos a asignarlas manualmente a cada hospital correspondiente en el DataFrame.

In [ ]:
coordenadas_manual = {
    "H.C.U. Valladolid": (41.655903, -4.718669),
    "H.U. Río Hortega": (41.628797, -4.711461),
}
df["Hospital"] = df["Hospital"].str.strip()

df["Latitud"] = df.apply(
    lambda row: coordenadas_manual[row["Hospital"]][0]
    if row["Hospital"] in coordenadas_manual
    else row["Latitud"],
    axis=1,
)
df["Longitud"] = df.apply(
    lambda row: coordenadas_manual[row["Hospital"]][1]
    if row["Hospital"] in coordenadas_manual
    else row["Longitud"],
    axis=1,
)

In [ ]:
# Filtrar filas donde Latitud o Longitud sean NaN
nan_coords = df[df[["Latitud", "Longitud"]].isna().any(axis=1)]

# Mostrar las filas con NaN en Latitud o Longitud
print(nan_coords[["Hospital", "Latitud", "Longitud"]])

###**COMENTARLO, CODIGO ACTUALIZADO**

Asignar el registro más cercano

In [ ]:
import pandas as pd
from datetime import timedelta
from meteostat import Stations, Hourly

# OBTENER ESTACIONES CERCANAS A CADA HOSPITAL ---
# Crear un DataFrame con hospitales únicos y sus coordenadas
hospitales_unicos = df[["Hospital", "Latitud", "Longitud"]].drop_duplicates().copy()


# Función para obtener la estación meteorológica más cercana
def obtener_estacion(lat, lon):
    stations = Stations()
    station = stations.nearby(lat, lon).fetch(1)
    return station.index[0] if not station.empty else None


# Asignar estación a cada hospital
hospitales_unicos["Estacion_hosp"] = hospitales_unicos.apply(
    lambda row: obtener_estacion(row["Latitud"], row["Longitud"]), axis=1
)

# Verificar que hospitales_unicos es un DataFrame válido
print("Tipo de hospitales_unicos:", type(hospitales_unicos))
print("Columnas en hospitales_unicos:", hospitales_unicos.columns.tolist())

# Hacer merge con df para agregar la estación más cercana a cada hospital
df = df.merge(hospitales_unicos, on=["Hospital", "Latitud", "Longitud"], how="left")

# AGRUPAR HORAS ÚNICAS POR HOSPITAL PARA OPTIMIZAR LA DESCARGA DE DATOS CLIMÁTICOS ---
hospitales_horas_unicas = df[["Estacion_hosp", "Fecha completa"]].drop_duplicates()

# Almacenar datos meteorológicos por estación y hora
resultados_clima = []

for estacion in hospitales_horas_unicas["Estacion_hosp"].dropna().unique():
    fechas_unicas = hospitales_horas_unicas[
        hospitales_horas_unicas["Estacion_hosp"] == estacion
    ]["Fecha completa"]

    # Definir rango de fechas a descargar
    fecha_inicio = fechas_unicas.min()
    fecha_fin = fechas_unicas.max() + timedelta(days=1)

    # Obtener datos meteorológicos solo para esas fechas y estación
    datos_clima = Hourly(estacion, fecha_inicio, fecha_fin).fetch()

    if not datos_clima.empty:
        datos_clima = datos_clima[["temp", "prcp"]].reset_index()
        datos_clima["Ha llovido"] = (datos_clima["prcp"] > 0).astype(int)
        datos_clima["Estacion_hosp"] = estacion
        resultados_clima.append(datos_clima)

# Unir todos los datos meteorológicos en un solo DataFrame
clima_df = pd.concat(resultados_clima, ignore_index=True)

# LIMPIEZA Y ORDENADO DE DATOS PARA MERGE ---
# Convertir fechas a datetime
df["Fecha completa"] = pd.to_datetime(df["Fecha completa"], errors="coerce")
clima_df["time"] = pd.to_datetime(clima_df["time"], errors="coerce")

# Eliminar valores NaN en claves principales
df = df.dropna(subset=["Fecha completa", "Estacion_hosp"])
clima_df = clima_df.dropna(subset=["time", "Estacion_hosp"])

# Eliminar duplicados en df
df = df.drop_duplicates(subset=["Estacion_hosp", "Fecha completa"])

# Eliminar duplicados en clima_df
clima_df = (
    clima_df.groupby(["Estacion_hosp", "time"])
    .agg({"temp": "mean", "Ha llovido": "max"})
    .reset_index()
)

# Ordenar ambos DataFrames para el merge_asof
df = df.sort_values(by=["Estacion_hosp", "Fecha completa"]).reset_index(drop=True)
clima_df = clima_df.sort_values(by=["Estacion_hosp", "time"]).reset_index(drop=True)

# REALIZAR EL MERGE ASOF ---
df = pd.merge_asof(
    df,
    clima_df[["time", "temp", "Ha llovido", "Estacion_hosp"]],
    left_on="Fecha completa",
    right_on="time",
    by="Estacion_hosp",
    direction="nearest",
)

# Eliminar la columna 'time' si ya no es necesaria
df = df.drop(columns=["time"])

# ✅ Mostrar resultados finales
print(df[["Hospital", "Fecha completa", "temp", "Ha llovido"]])


In [ ]:
df["Ha llovido"].unique()

####**3. Estudio de Nan.**

Contamos la cantidad de datos presentes en cada columna para identificar posibles datos faltantes.

In [ ]:
df.count()

Observamos que no todas las columnas contienen la misma cantidad de datos, por lo que procederemos a identificar qué variables presentan valores nulos (NaN). Esto nos permitirá decidir si estos valores faltantes impactarán en nuestro análisis y, por lo tanto, si será necesario eliminarlos, o si podrán ser ignorados sin comprometer los resultados de nuestra investigación.

Por ahora, no sabemos cómo pueden influir los valores nulos en variables como la procedencia y la zona básica de salud, por lo que no eliminaremos esas filas en esta etapa. En cuanto a las variables como edad y sexo, estas podrían tener un impacto significativo en nuestro análisis, por lo que se evaluarán más adelante antes de tomar una decisión definitiva.

In [ ]:
missing_data = df.isna().sum()

plt.figure(figsize=(10, 6))
missing_data.plot(
    kind="bar", color="#123456", edgecolor="black"
)  # Ajusta el color aquí
plt.title("Datos Faltantes por Columna", fontsize=16)
plt.xlabel("Columnas", fontsize=12)
plt.ylabel("Cantidad de Datos Faltantes", fontsize=12)
plt.xticks(rotation=45, ha="right", fontsize=10)
plt.grid(axis="y", alpha=0.7, linestyle="--")
plt.tight_layout()
plt.show()


In [ ]:
na_por_columna = df.isna().sum()

na_totales = df.isna().sum().sum()

print("Valores NaN por columna:")
print(na_por_columna)
print("\nNúmero total de valores NaN en el DataFrame:", na_totales)

Vamos a analizar si los datos faltantes coinciden entre las columnas "Zona Básica de Salud" y "Urbano", así como entre "Edad" y "Hombre". Más adelante, revisaremos los datos faltantes en "Nivel de Triaje".

In [ ]:
na_coinciden = (df["Zona Básica de Salud"].isna() == df["Urbano"].isna()).all()

print("¿Coinciden los NaN entre 'Zona Básica de Salud' y 'Urbano'?", na_coinciden)

nan_coinciden = (df["Edad"].isna() == df["Hombre"].isna()).all()

print("¿Coinciden los NaN entre 'Edad' y 'Hombre'?", nan_coinciden)

Hemos comprobado que coinciden los datos faltantes, ahora evaluaremos si los valores de Zona Básica de Salud y Urbano pueden derivarse directamente del hospital, lo que podría hacerlas redundantes.

In [ ]:
zona_ambito_hospital = df.groupby(["Área", "Hospital"])[
    ["Zona Básica de Salud", "Urbano"]
].nunique()
print("Niveles únicos por hospital:\n", zona_ambito_hospital)

No es posible deducir la Zona Básica de Salud (ZBS) ni el ámbito de procedencia únicamente a partir del hospital, ya que un mismo hospital atiende a pacientes provenientes de diferentes ZBS y ámbitos. Tampoco podemos determinarlo exclusivamente a partir del área. Sin embargo, este análisis nos permite identificar qué hospitales están asociados a cada área, lo cual es útil para comprender mejor la distribución geográfica de la atención hospitalaria.

También vamos a ver como se asocian las áreas con cada provincia.

In [ ]:
provincias_analisis = df.groupby("Provincia").agg({"Área": lambda x: x.unique()})

provincias_analisis = provincias_analisis.reset_index()

provincias_analisis.columns = ["Provincia", "Áreas"]

provincias_analisis

In [ ]:
duplicados = df.duplicated()

total_duplicados = duplicados.sum()

print(f"Total de filas completamente duplicadas: {total_duplicados}")

if total_duplicados > 0:
    print("Filas duplicadas:")
    print(df[duplicados])
else:
    print("No hay filas completamente duplicadas en el DataFrame.")


Para decidir si se deben eliminar los valores faltantes en *Nivel de Triaje*, agruparemos la cantidad de estos valores por mes con el objetivo de identificar posibles patrones temporales en los datos faltantes. Este análisis permitirá determinar si la ausencia de datos está concentrada en ciertos periodos específicos o si se distribuye de manera uniforme a lo largo del tiempo. Según los resultados, se evaluará si la eliminación de estos registros podría afectar la representatividad del análisis o si sería más adecuado considerar otras estrategias, como la imputación de datos.

In [ ]:
df["Año_Mes"] = df["Fecha completa"].dt.to_period("M")

nan_triaje_por_mes = df[df["Nivel de triaje"].isna()].groupby("Año_Mes").size()

# Contar el total de registros por mes para calcular proporciones
total_por_mes = df.groupby("Año_Mes").size()

proporcion_nan = (nan_triaje_por_mes / total_por_mes) * 100

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Cantidad de NaN por mes
nan_triaje_por_mes.plot(kind="bar", color="#123456", alpha=1, ax=axes[0])
axes[0].set_title('Cantidad de valores faltantes en "Nivel de Triaje" por mes')
axes[0].set_xlabel("Mes")
axes[0].set_ylabel("Cantidad de NaN")
axes[0].tick_params(axis="x", rotation=45)

# Gráfico 2: Proporción de NaN por mes
proporcion_nan.plot(kind="bar", color="#123456", alpha=0.7, ax=axes[1])
axes[1].set_title('Proporción de valores faltantes en "Nivel de Triaje" por mes (%)')
axes[1].set_xlabel("Mes")
axes[1].set_ylabel("Proporción de NaN (%)")
axes[1].tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()


A través de estos gráficos, podemos observar una mejora en la calidad de los registros de los datos. A inicios de 2021, la cantidad de valores faltantes superó los 20,000 en algunos meses, mientras que hacia finales de 2023 esta cantidad disminuyó a menos de 8,000.

Dado este comportamiento, la mejor opción podría ser eliminar los registros faltantes correspondientes a los meses de 2021, con el fin de evitar sesgos derivados de datos incompletos durante ese periodo.

En cuanto a los datos faltantes de 2022, dado que la proporción es significativamente menor y se observa una estabilización en la calidad del registro, podríamos optar por mantenerlos, imputarlos o eliminarlos, dependiendo de la importancia de estos valores en el análisis y del impacto que su ausencia pueda tener en los resultados.

####**4. Análisis de pacientes por año.**

En esta sección analizaremos cómo se distribuyen las distintas variables. En primer lugar, nos interesa examinar cómo se distribuyen los datos a lo largo del tiempo. Para ello, creamos un gráfico que muestra la cantidad de los datos por año.

Estudiar esto es relevante debido  debido al estado de alarma por COVID-19. Este análisis nos permitirá identificar si los datos de ese año podrían introducir sesgos en nuestros resultados y, en caso necesario, considerar estrategias para manejarlos adecuadamente.

In [ ]:
df["Fecha de atención"] = pd.to_datetime(df["Fecha de atención"], errors="coerce")

df["Año"] = df["Fecha de atención"].dt.year

conteo_por_año = df["Año"].value_counts().sort_index()

plt.figure(figsize=(10, 6))
plt.bar(conteo_por_año.index, conteo_por_año.values, color="#123456")
plt.title("Cantidad de datos por año", fontsize=16)
plt.xlabel("Año", fontsize=14)
plt.ylabel("Cantidad de datos", fontsize=14)
plt.xticks(conteo_por_año.index, rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

En 2021 se observa una menor cantidad de datos, lo que podría indicar que durante ciertos meses no se registraron datos en nuestra base de datos. Procederemos a comprobarlo, ya que inicialmente se esperaba un resultado contrario.

In [ ]:
df["Fecha de atención"] = pd.to_datetime(df["Fecha de atención"], errors="coerce")

plt.figure(figsize=(12, 8))

for year, color in zip([2021, 2022, 2023], ["red", "blue", "green"]):
    # Filtrar los datos del año
    datos_por_semana = (
        df[df["Fecha de atención"].dt.year == year]
        .groupby(df["Fecha de atención"].dt.isocalendar().week)
        .size()
    )

    plt.scatter(
        datos_por_semana.index,
        datos_por_semana.values,
        label=f"Año {year}",
        color=color,
        s=50,
        alpha=0.7,
    )

# Personalización del gráfico
plt.title("Cantidad de datos por semana para cada año", fontsize=16)
plt.xlabel("Semana del año", fontsize=14)
plt.ylabel("Cantidad de datos", fontsize=14)
plt.xticks(range(1, 54))  # Semanas de 1 a 53
plt.legend(title="Año", fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()


Verificamos que, efectivamente, durante todas las semanas del año 2021 se han registrado datos en nuestra base de datos. Sin embargo, para entender mejor la disminución en la cantidad total de datos de ese año, vamos a analizar si todos los hospitales tuvieron registros consistentes durante 2021. Esto nos permitirá identificar si la reducción de datos se debe a la falta de registros en ciertos hospitales o a algún otro motivo relacionado con la recopilación de información.

In [ ]:
# Filtrar hospitales sin valores nulos
hospitales_2021 = sorted(
    df.loc[df["Fecha de atención"].dt.year == 2021, "Hospital"].dropna().unique()
)
hospitales_totales = sorted(df["Hospital"].dropna().unique())

print("Hospitales en 2021:")
print("\n".join(hospitales_2021))
print(f"\nTotal de hospitales en 2021: {len(hospitales_2021)}\n")

print("Hospitales totales:")
print("\n".join(hospitales_totales))
print(f"\nTotal de hospitales totales: {len(hospitales_totales)}")


In [ ]:
datos_faltantes_2021 = df[df["Fecha de atención"].dt.year == 2021].isna().sum()
print("Datos faltantes en 2021:\n", datos_faltantes_2021)

Vemos que hay registros en todos los hospitales durante 2021, por lo que descartamos que la falta de datos esté relacionada con la ausencia de registros en algún hospital específico. Sin embargo, sigue siendo extraño que, a pesar de la pandemia de COVID-19, haya un menor número de registros durante este año. Para explorar más a fondo este fenómeno, investigaremos cuál es el mes con menos pacientes en 2021 y trataremos de extraer conclusiones a partir de esta información.

Vamos a analizar la cantidad de pacientes que acudieron a urgencias por mes en cada año, con el objetivo de identificar a partir de qué mes los datos de 2021 comienzan a asemejarse a los de 2022 y 2023. Esto nos permitirá determinar desde qué punto podemos considerar que la base de datos es representativa y adecuada para su análisis sin verse afectada por posibles anomalías relacionadas con la pandemia de COVID-19.

In [ ]:
# Contar la cantidad de pacientes por mes en cada año
pacientes_por_mes_2021 = (
    df[df["Fecha de atención"].dt.year == 2021]["Fecha de atención"]
    .dt.month.value_counts()
    .sort_index()
)
pacientes_por_mes_2022 = (
    df[df["Fecha de atención"].dt.year == 2022]["Fecha de atención"]
    .dt.month.value_counts()
    .sort_index()
)
pacientes_por_mes_2023 = (
    df[df["Fecha de atención"].dt.year == 2023]["Fecha de atención"]
    .dt.month.value_counts()
    .sort_index()
)

# Crear un array con los meses del año (1 a 12)
meses = np.arange(1, 13)

# Crear el gráfico de líneas
plt.figure(figsize=(12, 6))
plt.plot(
    meses, pacientes_por_mes_2021, marker="o", linestyle="-", label="2021", color="blue"
)
plt.plot(
    meses, pacientes_por_mes_2022, marker="o", linestyle="-", label="2022", color="red"
)
plt.plot(
    meses,
    pacientes_por_mes_2023,
    marker="o",
    linestyle="-",
    label="2023",
    color="green",
)

# Configuración del gráfico
plt.title("Comparación de Pacientes por Mes (2021-2023)", fontsize=16)
plt.xlabel("Mes", fontsize=14)
plt.ylabel("Número de Pacientes", fontsize=14)
plt.xticks(
    meses,
    [
        "Enero",
        "Febrero",
        "Marzo",
        "Abril",
        "Mayo",
        "Junio",
        "Julio",
        "Agosto",
        "Septiembre",
        "Octubre",
        "Noviembre",
        "Diciembre",
    ],
    rotation=45,
)
plt.legend()
plt.grid(alpha=0.3)

# Mostrar el gráfico
plt.show()


Vemos que a partir de mayo los datos de 2021 comienzan a asemejarse a los de los años siguientes, lo que sugiere que a partir de ese mes la base de datos puede considerarse representativa y sin anomalías significativas relacionadas con la pandemia de COVID-19. Para asegurarnos de ello, realizaremos un análisis cuantitativo mediante la comparación de las diferencias porcentuales entre los datos de 2021 y los de 2022 y 2023.

A través de este análisis, podremos verificar si el punto de inflexión observado en el gráfico anterior se mantiene en una comparación más detallada, confirmando si mayo es efectivamente el mes a partir del cual los datos de 2021 pueden utilizarse sin sesgos.

In [ ]:
# Crear un DataFrame con los datos de pacientes por mes en cada año
datos_pacientes = pd.DataFrame(
    {
        "Mes": np.arange(1, 13),
        "2021": pacientes_por_mes_2021,
        "2022": pacientes_por_mes_2022,
        "2023": pacientes_por_mes_2023,
    }
)

# Calcular diferencias porcentuales con respecto a 2021
datos_pacientes["Dif_2021_vs_2022"] = (
    (datos_pacientes["2022"] - datos_pacientes["2021"]) / datos_pacientes["2022"]
) * 100
datos_pacientes["Dif_2021_vs_2023"] = (
    (datos_pacientes["2023"] - datos_pacientes["2021"]) / datos_pacientes["2023"]
) * 100

# Graficar la diferencia porcentual mes a mes
plt.figure(figsize=(12, 6))
plt.plot(
    datos_pacientes["Mes"],
    datos_pacientes["Dif_2021_vs_2022"],
    marker="o",
    linestyle="-",
    label="Diferencia 2021 vs 2022",
    color="red",
)
plt.plot(
    datos_pacientes["Mes"],
    datos_pacientes["Dif_2021_vs_2023"],
    marker="o",
    linestyle="-",
    label="Diferencia 2021 vs 2023",
    color="green",
)

# Configuración del gráfico
plt.axvline(x=5, color="black", linestyle="--", label="Mayo")  # Línea vertical en junio
plt.title("Diferencia Porcentual de Pacientes (2021 vs 2022 y 2023)", fontsize=16)
plt.xlabel("Mes", fontsize=14)
plt.ylabel("Diferencia Porcentual (%)", fontsize=14)
plt.xticks(
    np.arange(1, 13),
    [
        "Enero",
        "Febrero",
        "Marzo",
        "Abril",
        "Mayo",
        "Junio",
        "Julio",
        "Agosto",
        "Septiembre",
        "Octubre",
        "Noviembre",
        "Diciembre",
    ],
    rotation=45,
)
plt.legend()
plt.grid(alpha=0.3)

plt.show()


El gráfico confirma que en mayo se produce un punto de inflexión en la diferencia porcentual de pacientes entre 2021 y los otros dos años (2022 y 2023). Hasta abril, las diferencias son significativamente altas, indicando que la cantidad de registros en 2021 era considerablemente menor en comparación con los otros años. Sin embargo, a partir de mayo, estas diferencias disminuyen de manera constante hasta estabilizarse en valores más bajos, lo que sugiere que la tendencia de pacientes en 2021 comienza a asemejarse a la de 2022 y 2023.

Este análisis nos indica que los datos de 2021 anteriores a mayo podrían estar sesgados y no reflejar correctamente la demanda de urgencias debido a la pandemia. Por lo tanto, una decisión adecuada para trabajar con esta base de datos sería considerar únicamente los registros a partir de mayo de 2021 en adelante, asegurándonos así de que los datos utilizados sean más comparables y representativos de la realidad hospitalaria post-pandemia.

####**5. Estudio las diferentes variables**

Ahora vamos a analizar las variables para verificar que todas tengan valores coherentes y consistentes. Nos enfocaremos en la variable "Edad" como ejemplo, observando sus máximos, mínimos, media y otros estadísticos relevantes. Este análisis nos permitirá identificar y eliminar datos que no sean reales, como valores negativos, extremadamente altos o que no correspondan con expectativas razonables. Este proceso es esencial para garantizar la calidad y la integridad de los datos antes de realizar cualquier análisis más profundo o tomar decisiones basadas en ellos.

In [ ]:
df["Edad"].describe()

A través de este análisis, observamos que la media de edad de los pacientes es de 47 años. Sin embargo, también encontramos que el valor mínimo de edad es -1 año, lo cual no es lógico, y el máximo de edad es de 122 años. Al investigar un poco más, encontramos que la persona más longeva del mundo tiene 117 años, lo que indica que estos valores extremos no tienen sentido y son probablemente errores en los datos. Por lo tanto, procederemos a eliminar estos outliers para garantizar la calidad y coherencia de la información antes de continuar con el análisis.

In [ ]:
edades_fuera_de_rango = df[(df["Edad"] < 0) | (df["Edad"] > 117)]

edades_fuera_de_rango.head()

In [ ]:
df["Edad"].describe()

Una vez hemos limpiado la variable de edad, vamos a proceder a ver los dias de la semana

In [ ]:
dias_esperados = [
    "LUNES",
    "MARTES",
    "MIÉRCOLES",
    "JUEVES",
    "VIERNES",
    "SÁBADO",
    "DOMINGO",
]

dias_unicos = df["Día de la semana"].unique()

print("Días únicos en la columna:", dias_unicos)

dias_no_validos = [dia for dia in dias_unicos if dia not in dias_esperados]

if dias_no_validos:
    print("Días no válidos encontrados:", dias_no_validos)
else:
    print("Todos los días están escritos de forma adecuada.")

filas_dias_no_validos = df[df["Día de la semana"].isin(dias_no_validos)]

if not filas_dias_no_validos.empty:
    print("Filas con días no válidos:")
    print(filas_dias_no_validos.head())

A continuación, el código verifica que no existan fechas inválidas, como el 30 de febrero, asegurando que todas las fechas en el DataFrame sean correctas y correspondan a días reales dentro de cada mes.

In [ ]:
df["Fecha de atención"] = pd.to_datetime(df["Fecha de atención"], errors="coerce")

fechas_invalidas = df[df["Fecha de atención"].isna()]

if fechas_invalidas.empty:
    print("Todas las fechas son válidas.")
else:
    print("Fechas no válidas encontradas:")
    print(fechas_invalidas)

df["Día"] = df["Fecha de atención"].dt.day
df["Mes"] = df["Fecha de atención"].dt.month
df["Año"] = df["Fecha de atención"].dt.year


def verificar_dias_invalidos(df):
    problemas = []
    for año in df["Año"].unique():
        for mes in df["Mes"].unique():
            # Obtener último día del mes para el año dado
            ultimo_dia = calendar.monthrange(año, mes)[1]
            # Filtrar filas donde el día excede el último día del mes
            dias_invalidos = df[
                (df["Año"] == año) & (df["Mes"] == mes) & (df["Día"] > ultimo_dia)
            ]
            if not dias_invalidos.empty:
                problemas.append(dias_invalidos)
    return problemas


problemas_dias = verificar_dias_invalidos(df)

if problemas_dias:
    print("Se encontraron fechas con días fuera de rango:")
    for problema in problemas_dias:
        print(problema)
else:
    print("No se encontraron días fuera de rango.")


Ahora realizamos la misma verificación con las horas, asegurándonos de que todas estén en un formato válido.

In [ ]:
df["Hora"] = df["Hora"].astype(str)


# Función para verificar horas válidas
def es_hora_valida(hora):
    try:
        # Intentar convertir la hora al formato datetime.time
        pd.to_datetime(hora, format="%H:%M").time()
        return True
    except ValueError:
        return False


# Aplicar la función para identificar horas inválidas
df["Hora válida"] = df["Hora"].apply(es_hora_valida)

# Filtrar las horas inválidas
horas_invalidas = df[~df["Hora válida"]]

if horas_invalidas.empty:
    print("Todas las horas son válidas.")
else:
    print("Horas inválidas encontradas:")
    print(horas_invalidas[["Fecha de atención", "Hora"]])

# Si no hay problemas con las horas, convertirlas a datetime.time para análisis adicional
if horas_invalidas.empty:
    df["Hora"] = pd.to_datetime(df["Hora"], format="%H:%M").dt.time

# Mostrar estadística de las horas
if horas_invalidas.empty:
    print("Hora más temprana:", df["Hora"].min())
    print("Hora más tardía:", df["Hora"].max())

Estudiamos los niveles de triaje, primero veremos cuantos niveles de triaje tenemos. Esto nos serivira para ver si hay alguno que no concuerde

In [ ]:
print(df["Nivel de triaje"].unique())

In [ ]:
print(df["Nivel de triaje"].value_counts())

Ahora vamos a entrar más en detalle en los datos que hay dentro del Nivel de triaje, podemos ver la cantidad de datos, la media de la edad

In [ ]:
# Filtrar filas donde "Nivel de triaje" sea NaN
df_nivel_triaje_nan = df[df["Nivel de triaje"].isna()]

# Configurar Pandas para mostrar todo en una línea
pd.set_option(
    "display.expand_frame_repr", False
)  # Evita que se divida en varias líneas
pd.set_option("display.max_columns", None)  # Muestra todas las columnas sin truncarlas

print("Descripción de los datos:")
print(df_nivel_triaje_nan.describe().T)  # Transponer para verlo en línea

print("\nBase de datos:")
print(
    df_nivel_triaje_nan.head().to_string(index=False)
)  # Evita que se muestren los índices


Observamos que en los primeros datos se repiten el mismo año, ámbito de procedencia y zona de salud, por lo que vamos a verificar si el nivel de triaje "Desconocido" solo aparece cuando se cumple alguna de estas condiciones.

In [ ]:
print(df_nivel_triaje_nan["Año"].unique())

In [ ]:
print(df_nivel_triaje_nan["Urbano"].unique())

In [ ]:
print(df_nivel_triaje_nan["Zona Básica de Salud"].unique())

Ahora analizamos la variable 'Sexo' y observamos que hay más datos de mujeres que de hombres. Esta diferencia en la distribución de los datos debe ser considerada al momento de realizar el proyecto y extraer conclusiones, ya que podría influir en los resultados y en la interpretación de los análisis.

In [ ]:
genero_counts = df["Hombre"].value_counts(dropna=False)

# Crear los colores para cada categoría
colors = {1: "blue", 0: "pink", None: "gray"}

# Crear el gráfico de barras
plt.figure(figsize=(8, 6))
plt.bar(
    genero_counts.index.astype(str),
    genero_counts.values,
    color=[colors.get(x, "gray") for x in genero_counts.index],
)
plt.title("Distribución de Género en la Base de Datos", fontsize=16)
plt.xlabel("Género", fontsize=12)
plt.ylabel("Cantidad", fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis="y", alpha=0.3)

# Agregar etiquetas a las barras
for i, count in enumerate(genero_counts.values):
    plt.text(i, count + 100, str(count), ha="center", fontsize=10)

plt.show()


Una vez hemos analizado todas las variables y eliminado los datos que no eran útiles o que podían sesgar el estudio, procedemos a examinar la información en profundidad para identificar patrones y extraer conclusiones relevantes.

Ahora vamos a convertir tambien abinario el Ámbito de procedencia.

####**6. Eliminación de Nans**

In [ ]:
# Filtrar eliminando solo los meses de enero a abril de 2021
df = df[
    ~(
        (df["Fecha de atención"].dt.year == 2021)
        & (df["Fecha de atención"].dt.month < 5)
    )
]

# Filtrar solo los registros con edades válidas (entre 0 y 117 años)
df = df[(df["Edad"] >= 0) & (df["Edad"] <= 117)]

# Eliminamos las filas donde Nivel de triaje es Nan
df = df.dropna(subset=["Nivel de triaje"])

##Análisis de datos


In [ ]:
df.head()

Vamos a analizar la distribución de los pacientes por mes para identificar posibles ciclos, tendencias o patrones en la cantidad de atenciones registradas. Esto nos permitirá detectar si hay días con mayor afluencia de pacientes, estacionalidades recurrentes o tendencias a lo largo del tiempo.

In [ ]:
# Extraer el año y el mes para clasificar los datos
df["Año"] = df["Fecha de atención"].dt.year
df["Mes"] = df["Fecha de atención"].dt.month

# Agrupar por mes y año
pacientes_por_mes = df.groupby(["Mes", "Año"]).size().unstack()

# Crear el gráfico con líneas superpuestas por año
plt.figure(figsize=(12, 6))

for year in pacientes_por_mes.columns:
    plt.plot(
        pacientes_por_mes.index,
        pacientes_por_mes[year],
        marker="o",
        linestyle="-",
        label=str(year),
    )

plt.xlabel("Mes", fontsize=12)
plt.ylabel("Número de Pacientes", fontsize=12)
plt.title("Comparación de Pacientes por Mes en Distintos Años", fontsize=16)
plt.legend(title="Año")
plt.grid(True, alpha=0.3)
plt.xticks(
    ticks=range(1, 13),
    labels=[
        "Ene",
        "Feb",
        "Mar",
        "Abr",
        "May",
        "Jun",
        "Jul",
        "Ago",
        "Sep",
        "Oct",
        "Nov",
        "Dic",
    ],
)
plt.show()


Observamos que febrero suele ser el mes con menor cantidad de pacientes, probablemente debido a que tiene menos días. También detectamos una caída en el número de pacientes en junio y septiembre. La disminución en junio podría estar relacionada con el inicio del verano, cuando muchas personas comienzan sus vacaciones y la actividad en las ciudades disminuye, lo que podría reducir la afluencia a los servicios de urgencias. En septiembre, la vuelta a la rutina puede limitar el tiempo disponible para acudir a urgencias en casos de enfermedades leves.  

A continuación, generaremos la misma gráfica, pero con los datos normalizados para evitar que la menor cantidad de días en febrero influya en la interpretación. Ajustaremos la proporción de pacientes para que todos los meses se consideren de 30 días, permitiéndonos extraer conclusiones más precisas.

In [ ]:
# Contar el número de pacientes por mes en cada año
pacientes_por_mes = df.groupby(["Año", "Mes"]).size()

# Obtener el número de días únicos en cada mes del conjunto de datos
dias_en_cada_mes = df.groupby(["Año", "Mes"])["Fecha de atención"].nunique()

# Ajustar los valores como si cada mes tuviera 30 días
pacientes_normalizados = (pacientes_por_mes / dias_en_cada_mes) * 30

# Crear el gráfico con líneas superpuestas por año
plt.figure(figsize=(12, 6))

for year in pacientes_normalizados.index.get_level_values(0).unique():
    datos_anuales = pacientes_normalizados.xs(
        year, level=0
    )  # Extraer datos de cada año
    plt.plot(
        datos_anuales.index,
        datos_anuales.values,
        marker="o",
        linestyle="-",
        label=str(year),
    )

plt.xlabel("Mes", fontsize=12)
plt.ylabel("Número de Pacientes (Normalizado a 30 días)", fontsize=12)
plt.title(
    "Comparación de Pacientes por Mes en Distintos Años (Normalizado)", fontsize=16
)
plt.legend(title="Año")
plt.grid(True, alpha=0.3)
plt.xticks(
    ticks=range(1, 13),
    labels=[
        "Ene",
        "Feb",
        "Mar",
        "Abr",
        "May",
        "Jun",
        "Jul",
        "Ago",
        "Sep",
        "Oct",
        "Nov",
        "Dic",
    ],
)
plt.show()


In [ ]:
pd.set_option("display.expand_frame_repr", False)  # Evita la división de columnas

# Contar el número de pacientes por mes en cada año
pacientes_por_mes = df.groupby(["Año", "Mes"]).size()

# Obtener el número de días únicos en cada mes del conjunto de datos
dias_en_cada_mes = df.groupby(["Año", "Mes"])["Fecha de atención"].nunique()

# Ajustar los valores como si cada mes tuviera 30 días
pacientes_normalizados = (pacientes_por_mes / dias_en_cada_mes) * 30

# Encontrar el mes con más y menos pacientes por cada año
mes_max_pacientes = pacientes_normalizados.groupby(level=0).idxmax()
mes_min_pacientes = pacientes_normalizados.groupby(level=0).idxmin()

# Obtener la cantidad de pacientes normalizados para cada mes máximo y mínimo
cantidad_max_pacientes = pacientes_normalizados.loc[mes_max_pacientes.values].values
cantidad_min_pacientes = pacientes_normalizados.loc[mes_min_pacientes.values].values

# Crear un DataFrame con los resultados y las cantidades correspondientes
resultados = pd.DataFrame(
    {
        "Año": mes_max_pacientes.index,
        "Mes con más pacientes": [
            m[1] for m in mes_max_pacientes.values
        ],  # Extraer solo el mes
        "Cantidad más pacientes (Normalizada)": cantidad_max_pacientes,
        "Mes con menos pacientes": [
            m[1] for m in mes_min_pacientes.values
        ],  # Extraer solo el mes
        "Cantidad menos pacientes (Normalizada)": cantidad_min_pacientes,
    }
)

# Mostrar resultados
print(resultados)


In [ ]:
# Ordenar los meses con más pacientes de mayor a menor cantidad
meses_ordenados = pacientes_normalizados.sort_values(ascending=False).reset_index()
meses_ordenados.columns = ["Año", "Mes", "Cantidad de Pacientes (Normalizada)"]

# Mostrar resultados
print(meses_ordenados)


In [ ]:
df["Día"] = df["Fecha de atención"].dt.day

# Contar el número de pacientes por día y mes en cada año
pacientes_por_dia_mes = df.groupby(["Año", "Mes", "Día"]).size()

# Encontrar el día y mes con menos pacientes para cada año
dia_mes_menos_pacientes = pacientes_por_dia_mes.groupby("Año").idxmin()
cantidad_menos_pacientes = pacientes_por_dia_mes.groupby("Año").min()

# Crear un DataFrame con los resultados
resultado_menos = pd.DataFrame(
    {
        "Día y Mes con menos pacientes": dia_mes_menos_pacientes,
        "Cantidad de pacientes": cantidad_menos_pacientes,
    }
)

# Mostrar los resultados
print(resultado_menos)

In [ ]:
dia_mes_mas_pacientes = pacientes_por_dia_mes.groupby("Año").idxmax()
cantidad_mas_pacientes = pacientes_por_dia_mes.groupby("Año").max()

resultado_mas = pd.DataFrame(
    {
        "Día y Mes con mas pacientes": dia_mes_mas_pacientes,
        "Cantidad de pacientes": cantidad_mas_pacientes,
    }
)

print(resultado_mas)

In [ ]:
# Asegurar que la columna de fecha esté en formato datetime
df["Fecha de atención"] = pd.to_datetime(df["Fecha de atención"], errors="coerce")

# Eliminar valores nulos en Edad y Nivel de Triaje
df = df.dropna(subset=["Edad", "Nivel de triaje"])

# Convertir Nivel de Triaje a string para evitar errores de categorización
df["Nivel de triaje"] = df["Nivel de triaje"].astype(str)

# Crear un boxplot para visualizar la distribución de edades por nivel de triaje
plt.figure(figsize=(12, 6))
df.boxplot(
    column="Edad", by="Nivel de triaje", grid=False, vert=False, patch_artist=True
)
plt.title("Distribución de Edades por Nivel de Triaje")
plt.xlabel("Edad")
plt.ylabel("Nivel de Triaje")
plt.suptitle("")  # Eliminar el título automático de pandas
plt.show()


##Entrenamiento de datos

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df["Nivel de triaje"] = pd.to_numeric(
    df["Nivel de triaje"], errors="coerce"
)  # Convierte strings a números y NaN a NaN
df["Nivel de triaje"] = df["Nivel de triaje"].astype(int)  # Convertir a entero

df["Edad"] = pd.to_numeric(df["Edad"], errors="coerce")
df["Edad"] = df["Edad"].astype(int)

In [ ]:
# Lista de columnas relacionadas con el tiempo que queremos excluir
columnas_excluir = [
    "Hora del día",
    "Día del año",
    "Mes del año",
    "Hora_sin",
    "Hora_cos",
    "Día_sin",
    "Día_cos",
    "Mes_sin",
    "Mes_cos",
    "Año",
    "Día",
    "Mes",
]

# Seleccionar solo las columnas numéricas, excluyendo las de tiempo
df_numerico_filtrado = df.select_dtypes(include=["number"]).drop(
    columns=columnas_excluir, errors="ignore"
)

# Calcular la matriz de correlación
correlaciones_filtradas = df_numerico_filtrado.corr()

# Visualizar la matriz de correlación sin las variables de tiempo
plt.figure(figsize=(10, 6))
sns.heatmap(
    correlaciones_filtradas, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5
)
plt.title("Matriz de Correlación sin Variables de Tiempo")
plt.show()

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# --- 1️⃣ DEFINIR VARIABLES ---
target = "Nivel de triaje"  # Variable objetivo
X = df.drop(columns=[target])  # Variables predictoras
y = df[target]  # Valores de la variable objetivo

# --- 2️⃣ ELIMINAR COLUMNAS NO NUMÉRICAS / CONVERTIR FORMATO ---
# Revisar si hay columnas de tipo datetime
datetime_cols = X.select_dtypes(include=["datetime"]).columns
print("Columnas datetime encontradas:", datetime_cols.tolist())

# Convertir fechas a timestamp (segundos desde 1970)
for col in datetime_cols:
    X[col] = X[col].astype("int64") // 10**9

# Revisar si hay columnas categóricas (no numéricas)
non_numeric_cols = X.select_dtypes(exclude=["number"]).columns
print("Columnas categóricas encontradas:", non_numeric_cols.tolist())

# Convertir variables categóricas a numéricas con One-Hot Encoding
if len(non_numeric_cols) > 0:
    X = pd.get_dummies(X, columns=non_numeric_cols)

# --- 3️⃣ DIVIDIR EN CONJUNTOS DE ENTRENAMIENTO Y PRUEBA ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- 4️⃣ CREAR Y ENTRENAR EL MODELO ---
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# --- 5️⃣ HACER PREDICCIONES ---
y_pred = modelo.predict(X_test)

# --- 6️⃣ EVALUAR EL MODELO ---
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")
